# More on Types 

In [1]:
struct Color{T} <: Number
    R::T
    G::T
    B::T
end

In [2]:
c1=Color{Int}(1,2,3)

Color{Int64}(1, 2, 3)

In [3]:
c2=Color{String}("1","2","3")

Color{String}("1", "2", "3")

In [4]:
typeof(c1)

Color{Int64}

In [5]:
typeof(c2)

Color{String}

In [6]:
typeof(c1)==typeof(c2)

false

In [7]:
subtypes(Number)

3-element Array{Any,1}:
 Color
 Complex
 Real

In [8]:
Color{Int} <: Integer

false

In [9]:
Color{Int} <: Number

true

# Binary file reading

In [10]:
open("../data/TEST.DAT") do io
    read(io,Float32) |> print
end

-2.3884733

### A syntactical interlude 

In [11]:
function myfunction(f,x,y,z)
    f(x)+f(y)+f(z)
end

myfunction (generic function with 1 method)

In [12]:
myfunction(x->x^2,1,2,2)

9

In [13]:
myfunction(1,2,2) do x
    (x+5)
end

20

### back to the main topic

In [14]:
open("../data/TEST.DAT") do io
    read(io,Float32) |> println
    read(io,Float32) |> println
end

-2.3884733
49.441597


In [15]:
"49.441597" |> sizeof

9

This files containts and number of records 
* long, lat (32bit float)
* speed (16bit Int) 
* COG (16 UInt)
* time (32 Int)
* data (elements 32 bit int) 6x600 (ch1, ch2....ch6)x600

In [16]:
r_size=(32*2+16+16+32+6*600*32)÷8

14416

In [17]:
n_rec=filesize("../data/TEST.DAT")÷r_size

636

In [18]:
struct RecHeader
    long::Float32
    lat::Float32
    speed::Int16
    COG::UInt16
    time::Int32
end

In [19]:
r=RecHeader(43,53,12,21,89)

RecHeader(43.0f0, 53.0f0, 12, 0x0015, 89)

In [20]:
isbits(r)

true

In [21]:
struct RecHeaderExtended 
    long::Float32
    lat::Float32
    speed::Int16
    COG::UInt16
    time::Int32
    data::Matrix{Int32}
end

In [22]:
r=RecHeaderExtended(43,53,12,21,89,rand(Int32,600,6))

RecHeaderExtended(43.0f0, 53.0f0, 12, 0x0015, 89, Int32[1107358592 -1761135562 … 1097649913 774870181; 1854514384 -1038358468 … -284969805 -1589639252; … ; -1874322762 -1460206273 … -1810831345 -1543421872; -201348890 -902616549 … -529091103 1003438029])

In [23]:
a=Array{RecHeader}(undef,2)
open("../data/TEST.DAT") do io
    read!(io, a) |> println
end

RecHeader[RecHeader(-2.3884733f0, 49.441597f0, 522, 0x0ced, 23269524), RecHeader(5.93531f-40, 5.97339f-40, -20017, 0x0006, 604280)]


In [24]:
open("../data/TEST.DAT") do io
    read!(io, Array{RecHeader}(undef,1)) |> println
end

RecHeader[RecHeader(-2.3884733f0, 49.441597f0, 522, 0x0ced, 23269524)]


In [25]:
a[2]

RecHeader(5.93531f-40, 5.97339f-40, -20017, 0x0006, 604280)

In [26]:
a=Array{RecHeaderExtended}(undef,2)

2-element Array{RecHeaderExtended,1}:
 #undef
 #undef

In [27]:
open("../data/TEST.DAT") do io
    read!(io, a) |> println
end

ErrorException: The IO stream does not support reading objects of type RecHeaderExtended.

In [28]:
function Base.read(io::IOStream, ::Type{RecHeaderExtended})
    long=read(io,Float32)
    lat=read(io,Float32)
    speed=read(io,Int16)
    COG=read(io,UInt16)
    time=read(io,Int32)
    data=Array{Int32}(undef,6,600)
    read!(io,data)
    RecHeaderExtended(long,lat,speed,COG,time, data)
end

In [29]:
open("../data/TEST.DAT") do io
    read(io, RecHeaderExtended) 
end

RecHeaderExtended(-2.3884733f0, 49.441597f0, 522, 0x0ced, 23269524, Int32[423558 423535 … 430269 430865; 426275 430871 … 432497 430643; … ; 442844 441355 … 442458 442234; 428231 428297 … 430647 431706])

In [30]:
my_data=Vector{RecHeaderExtended}(undef,n_rec)
read!("../data/TEST.DAT",my_data);

In [31]:
my_data;

In [32]:
struct ChAcc{N} <: AbstractArray{Int32,1}
    d::Vector{RecHeaderExtended}
end

https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array-1

### Exercise 1

Create a wrapper around `my_data` to be able to acess a particular channel as and array

In [ ]:
channel_1=ChAcc{1}(my_data)
plot(channel_1)
length(channel_1)==600*632 # should be true

### Exercise 2

Modify the function `Base.read(io::IOStream, ::Type{RecHeaderExtended})` to be more concices.. 
    should make use of field info.

### Exercise 3

Test the performance of column based vs. row based representation of channels 